# Data integration with ICESat-2 - Part II

```{admonition} Learning Objectives
**Goals**
- Access NSIDC data sets and acquire IS-2 using icepyx
- Analyze point and raster data together with IS-2
- Advanced visualizations of multiple datasets
```
☝️ This formatting is a Jupyter Book [admonition](https://jupyterbook.org/content/content-blocks.html#notes-warnings-and-other-admonitions), that uses a custom version of Markdown called {term}`MyST`

## Libraries need to add:
* nisardev
* grimpfunc

# Comparing ICESat-2 Data with Other Datasets
---

Last time, we did a bit of work to add ICESat-2 and Operation Icebridge data to Pandas Dataframes. We only covered the basic operations that you can do with Pandas, so today we are going to do a more thorough analysis of the data here.

Since we already downloaded the ICESat-2/ATM files of interest, we are not going to use icepyx just yet - we will go ahead and reload the data from yesterday.

(Prompt) I forgot how to load the ICESat-2 data from a .h5 file. What do I need to do?

(Prompt) I also forgot how to load the ATM data. How do I read the CSV?

In [ ]:
import h5py
import pandas as pd
import numpy as np

We established last time that ATM aligns best with the central ICESat-2 beams, particularly the central strong beam (GT2L). Let's see if that is reflected in the elevation profiles...

In [ ]:
# Load the ICESat-2 data
is2_file = 'processed_ATL06_20190420093051_03380303_005_01_full.h5'
with h5py.File(is2_file, 'r') as f:
    is2_gt2r = pd.DataFrame(data={'lat': f['gt2r/land_ice_segments/latitude'][:],
                                  'lon': f['gt2r/land_ice_segments/longitude'][:],
                                  'elev': f['gt2r/land_ice_segments/h_li'][:]}) # Central weak beam
    is2_gt2l = pd.DataFrame(data={'lat': f['gt2l/land_ice_segments/latitude'][:],
                                  'lon': f['gt2l/land_ice_segments/longitude'][:],
                                  'elev': f['gt2l/land_ice_segments/h_li'][:]}) # Central strong beam
    
# Load the ATM data
atm_file = 'ILATM2_20190506_151600_smooth_nadir3seg_50pt.csv'
atm_l2 = pd.read_csv(atm_file)

# Look only at the nadir track
atm_l2 = atm_l2[atm_l2['Track_Identifier']==0]

# Change the longitudes to be consistent with ICESat-2
atm_l2['Longitude(deg)'] -= 360

In [ ]:
# Subset the ICESat-2 data to the ATM latitudes
is2_gt2r = is2_gt2r[(is2_gt2r['lat']<atm_l2['Latitude(deg)'].max()) & (is2_gt2r['lat']>atm_l2['Latitude(deg)'].min())]
is2_gt2l = is2_gt2l[(is2_gt2l['lat']<atm_l2['Latitude(deg)'].max()) & (is2_gt2l['lat']>atm_l2['Latitude(deg)'].min())]

In [ ]:
# Make a 2D plot of along-track surface height
import matplotlib.pyplot as plt

%matplotlib widget
plt.plot(is2_gt2r['lat'], is2_gt2r['elev'], label='gt2r')
plt.plot(is2_gt2l['lat'], is2_gt2l['elev'], label='gt2l')
plt.plot(atm_l2['Latitude(deg)'], atm_l2['WGS84_Ellipsoid_Height(m)'], label='atm')
plt.xlabel('latitude')
plt.ylabel('elevation [m]')
plt.xlim([69.185, 69.275])
plt.ylim([100, 550])
plt.legend()
plt.show()

Sure enough, GT2L and ATM match very well! Since they are very close to each other, we can do a quick accuracy assessment between the two.

The ATM DataFrame is larger than the ICESat-2 dataframe, so we're going to apply a simple spline interpolant to downscale the ICESat-2 data.

In [ ]:
from scipy.interpolate import splrep,splev

# Apply a spline interpolant to the ICESat-2 data
spl = splrep(is2_gt2l['lat'], is2_gt2l['elev'], s=0)
is2_spl = splev(atm_l2['Latitude(deg)'], spl, der=0)

# Calculate GT2L bias and add it to the ATM DataFrame
atm_l2['bias'] = atm_l2['WGS84_Ellipsoid_Height(m)'] - is2_spl

# Plot the bias curve
plt.plot(atm_l2['Latitude(deg)'], atm_l2['bias'])
#plt.plot(atm_l2['Latitude(deg)'], atm_l2['WGS84_Ellipsoid_Height(m)'])
#plt.plot(atm_l2['Latitude(deg)'], is2_spl)
plt.xlabel('latitude')
plt.ylabel('bias [m]')
plt.xlim([69.2, 69.26])
plt.ylim([-20, 20])
plt.show()

print('Mean bias: %s m' %(atm_l2['bias'].mean()))

Through some relatively simple operations, we found that ATM and ICESat-2 differ by ~0.33 m on average. Between this plot and the elevation plot above, what do you think might be causing some of the differences?

We will revisit ICESat-2 and ATM near the end of this tutorial. Now, we are going to look at ice velocities and flow lines from the GRIMP project.

# Plot Flowines Using Remote Greenland Ice Mapping Project Data
---

This notebook demonstrates how Greenland Ice Mapping Project can be remotely accessed to create plots along flowlines from [Felikson et al., 2020](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2020GL090112), which are archived on [Zenodo](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2020GL090112). The copies of the shapefiles included in this repository were downloaded in late January 2022. The notebook works with a specific set of glacier ids but is easily modified to plot results for other glaciers in the flowline data set.

In [ ]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2
import nisardev as nisar
import os
import matplotlib.colors as mcolors
import grimpfunc as grimp
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import numpy as np
import xarray as xr
import importlib
import requests
import pyproj
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import panel
from dask.diagnostics import ProgressBar
ProgressBar().register()
panel.extension()

In [ ]:
!pip install  git+https://github.com/fastice/grimpfunc.git@master;
!pip install git+https://github.com/fastice/nisardev.git@main;

## NSIDC Login

For remote access to the velocity data at NSIDC, run these cells to login with your NASA EarthData Login (see  [NSIDCLoginNotebook](https://github.com/fastice/GRiMPNotebooks/blob/master/NSIDCLoginNotebook.ipynb) for further details). These cells can skipped if all data are being accessed locally. First define where the cookie files need for login are saved.

In [ ]:
env = dict(GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/.grimp_download_cookiejar.txt'),
            GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/.grimp_download_cookiejar.txt'))
os.environ.update(env)

Now enter credentials. If previous valid cookie exists, no user input is needed.

In [ ]:
myLogin = grimp.NASALogin()
myLogin.view()

## Read Shapefiles

In the examples presented here we will use glaciers 1 & 2  in the Felikson data base, [Felikson et al., 2020](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2020GL090112), which were retrieved from [Zenodo](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2020GL090112). 

Each glacier's flowlines are used to create `grimp.Flowlines` instances, which are saved in a dictionary, `myFlowlines` with glacier id: '0001' and  '0002'. 

Each `Flowlines` read a set of flowlines for each glacier and stores in a dictionary of `myFlowlines.flowlines`. The code to do this looks something like:

```
    flowlines = {}
    shapeTable = gpd.read_file(shapefile)
    for index, row in shapeTable.iterrows():  # loop over features
        fl = {}  # New Flowline
        fl['x'], fl['y'] = np.array([c for c in row['geometry'].coords]).transpose()
        fl['d'] = computeDistance(fl['x'], fl['y'])
        flowlines[row['flowline']] = fl
 ```
For further detail, see the full [class defintion](https://github.com/fastice/grimpfunc/blob/master/grimpfunc/Flowlines.py)

To limit the plots to the downstream regions, the flowlines are all truncated to a `length` of 50km. 

Within each myFlowines entry (a `grimp.Flowlines` instance), the individual flowlines are maintained as a dictionary `myFlowlines['glacierId'].flowlines`.

In [ ]:
myShapeFiles = [f'./shpfiles/glacier000{i}.shp' for i in range(1, 3)] # Build list of shape file names
myFlowlines = {x[-8:-4]: grimp.Flowlines(shapefile=x, name=x[-8:-4], length=50e3) for x in myShapeFiles} 
myFlowlines

Each flowline is indexed as shown here:

In [ ]:
myFlowlines['0001'].flowlines.keys()

The data for the flow line is simple, just `x`, `y` polar stereographic coordinates (EPSG=3413) and the distance, `d`, from the start of the flowline.

In [ ]:
myFlowlines['0001'].flowlines['03'].keys()

These coordinates for a given index can be returned as `myFlowlines['0001'].xym(index='03')` or `myFlowlines['0001'].xykm(index='03')`, depending on whether m or km are preferred.

The area of interest can be defined as the union of the bounds for all of the flowlines computed as shown below along with the unique set of flowline IDs across all glaciers.

In [ ]:
myBounds = {'minx': 1e9, 'miny': 1e9, 'maxx': -1e9, 'maxy': -1e9}  # Initial bounds to force reset
flowlineIDs = []  # 
for myKey in myFlowlines:
    # Get bounding box for flowlines
    flowlineBounds = myFlowlines[myKey].bounds
    # Merge with prior bounds
    myBounds = myFlowlines[myKey].mergeBounds(myBounds, flowlineBounds)
    # Get the flowline ids
    flowlineIDs.append(myFlowlines[myKey].flowlineIDs())
# Get the unique list of flowlines ids (used for legends later)
flowlineIDs = np.unique(flowlineIDs)
print(myBounds)
print(flowlineIDs)

## Search Catalog for Velocity Data

We now need to locate velocity data from the GrIMP data set. For this excercise, we will focus on the annual velocity maps of Greenland. To do this, we will use the `grimp.cmrUrls` tool, which will do a GUI based search of NASA's Common Metadata Repository ([CMR](https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/cmr)). Search parameters can be passe directly to `initialSearch` method to perform the search.

In [ ]:
myUrls = grimp.cmrUrls(mode='nisar', verbose=True)  # nisar mode excludes image and tsx products and allows only one product type at a time
myUrls.initialSearch(product='NSIDC-0725')

The `verbose` flag causes the CMR search string to be printed. The search basically works by a) reading the parameters from the search panel (e.g., product, date, etc) and creating a search string, which returns the search result.

In [ ]:
response = requests.get('https://cmr.earthdata.nasa.gov/search/granules.json?provider=NSIDC_ECS&sort_key[]=start_date&sort_key[]='
                        'producer_granule_id&scroll=false&page_size=2000&page_num=1&short_name=NSIDC-0725&version=3&temporal[]='
                        '2000-01-01T00:00:01Z,2022-03-10T00:23:59&bounding_box[]=-75.00,60.00,-5.00,82.00&producer_granule_id[]='
                        '*&options[producer_granule_id][pattern]=true')
search_results = response.json()
search_results;

Under the hood, the `cmrUrls` code can filter the json to get a list of urls:

In [ ]:
myUrls.getURLS()

## Load the Velocity Data

GrIMP produces full Greenland velocity maps. Collectively, there are more than 400 full Greenland maps, totalling several hundred GB of data, which may be more than a user interested in a few glaciers wants to download and store on their laptop.  Fortunately using Cloud Optimized Geotiffs, which have the following properties:

- All the metadata is at the beginning of the file, allowing a single read to obtain the layout. 
- The data are tiled (i.e., stored as a series of blocks like a checkerboard) rather than as a line-by-line raster.  
- A consistent set of overview images (pyramids) are stored with the data.

While the velocity data are stored as multiple files at NSIDC, they can all be combined into a single `nisarVelSeries` instance, which has the following properties:

- Built on Xarray,
- Dask (arallel operations),
- Local and remote subsetting (Lazy Opens), and
- Subsets can be saved for later use

Before loading the data, we must setup the filename template. Specifically, we must put a '\*' where the band identifier would go and remove the trailing '.tif' extention. 

In [ ]:
urlNames = [x.replace('vv','*').replace('.tif','') for x in myUrls.getCogs()] # getCogs filters to ensure tif products
urlNames[0:5]

We can now create a `nisarVelocitySeries` object, which will create a large times series stack with all of the data.

In [ ]:
myVelSeries = nisar.nisarVelSeries() # Create Series
myVelSeries.readSeriesFromTiff(urlNames, url=True, readSpeed=False)  # readSpeed=False computes speed from vx, vy rather than downloading
myVelSeries.xr  # Add semicolon after to suppress output

For the annual data set, this step produces a ~7GB data sets, which expands to 370GB for the full 6-12-day data set. 

To avoid downloading unnessary data, the data can be subsetted using the bounding box from the flowlines. 

In [ ]:
myVelSeries.subSetVel(myBounds) # Apply subset
myVelSeries.subset # Add semicolon after to suppress output

The volume of the data set is now a far more manageable ~15MB, which is still located in the archive.  

With dask, operations can continue without downloading, until the data are finally needed to do something (e.g., create a plot).

If lots of operations are going to occur, however, it is best to download the data upfront. 

In [ ]:
myVelSeries.loadRemote() # Load the data to memory

## Overview Images

In the above, we picked out a small region for Greenland and downloaded a full-res data series. But in some cases, we may want the full image at reduced resolution (e.g., for an overview map). 

Here we can take advantage of the overviews to pull a single velocity map at reduced resolution (`overviewLevel=3`).

In [ ]:
urlNames[-1]

In [ ]:
myOverview = nisar.nisarVelSeries() # Create Series
myOverview.readSeriesFromTiff([urlNames[-1]], url=True, readSpeed=False, overviewLevel=3)  # readSpeed=False computes speed from vx, vy rather than downloading
myOverview.loadRemote()
myOverview.xr  # Add semicolon after to suppress output

## Display Flowlines and Velocity

In the next cell, we put the above pieces together.

Specifically, the flowlines over one of the velocity layers can be displayed as with the following block of code. 

In addition to plotting the flowline, a point 10 km along each flowline is plotted and saved for subsequent plots below.

In [ ]:
# set up figure and axis
fig, axes = plt.subplots(1, 2, figsize=(16,8))
#
# Create a dictionary for accumulating glacier points
glacierPoints = {}
# generate a color dict that spans all flowline ids, using method from a flowline instance
flowlineColors = list(myFlowlines.values())[0].genColorDict(flowlineIDs=flowlineIDs)
# Plot velocity maps
# Saturate at 2000 m/yr to preserve slow detail
myVelSeries.displayVelForDate('2020-01-01', ax=axes[0], labelFontSize=12, plotFontSize=9, titleFontSize=14, 
                              vmin=0, vmax=2000, units='km', scale='linear', colorBarSize='3%') 
myVelSeries.displayVelForDate('2020-01-01', ax=axes[1], labelFontSize=12, plotFontSize=9, titleFontSize=14, 
                              vmin=1, vmax=3000, units='km', scale='log', midDate=False, colorBarSize='3%')
# Plot location inset
height = 3
axInset = inset_axes(axes[0], width=height * myOverview.sx/myOverview.sy, height=height, loc=1)
myOverview.displayVelForDate(None, ax=axInset, vmin=1, vmax=3000, colorBar=False, scale='log', title='')
axInset.plot(*myVelSeries.outline(), color='r')
axInset.axis('off')
#
# Loop over each glacier and plot the flowlines
for glacierId in myFlowlines:
    # Plot the flowline Match units to the map
    myFlowlines[glacierId].plotFlowlineLocations(ax=axes[0], units='km', colorDict=flowlineColors)
    myFlowlines[glacierId].plotFlowlineLocations(ax=axes[1], units='km', colorDict=flowlineColors)
    # 
    myFlowlines[glacierId].plotGlacierName(ax=axes[0], units='km', color='w', fontsize=12,fontweight='bold', first=False)
    myFlowlines[glacierId].plotGlacierName(ax=axes[1], units='km', color='w', fontsize=12,fontweight='bold', first=False)
    #
    # Generates points 10km from downstream end of each flowline
    points10km = myFlowlines[glacierId].extractPoints(10, None, units='km')
    glacierPoints[glacierId] = points10km
    for key in points10km:
        axes[0].plot(*points10km[key], 'r.')
        axes[1].plot(*points10km[key], 'r.')
#
# Add legend
for ax in axes:
    # Create a dict of unique labels for legend
    h, l = ax.get_legend_handles_labels()
    by_label = dict(zip(l, h)) # will overwrite identical entries to produce uniqe values
    ax.legend(by_label.values(), by_label.keys(), title='Flowline ID', ncol=2, loc='lower left', fontsize=14)
#fig.tight_layout()

## Interpolation

A common function with the velocity date is interpolating data for plotting points or profiles, which can be easily done with the `nisarVelSeries.interp` method.

In [ ]:
#
# Using km
vx, vy, vv = myVelSeries.interp(*myFlowlines[glacierId].xykm(), units='km')
print(vx.shape, vx[0, 100], vy[0, 100], vv[0, 100])
#
# or units of meters
vx, vy, vv = myVelSeries.interp(*myFlowlines[glacierId].xym(), units='m')
print(vx.shape, vx[0, 100], vy[0, 100], vv[0, 100])
#
# or entirely different coordinate system
xytoll = pyproj.Transformer.from_crs(3413, 4326)
lat, lon = xytoll.transform(*myFlowlines[glacierId].xym())
vx, vy, vv = myVelSeries.interp(lat, lon, sourceEPSG=4326)
print(vx.shape, vx[0, 100], vy[0, 100], vv[0, 100])
# Or would prefer an xarray rather than nparray
result = myVelSeries.interp(*myFlowlines[glacierId].xykm(), units='km', returnXR=True)
result;

## Plot Central Flowlines at Different Times

This example will demonstrate plotting the nominally central flowline ('06') for each of the six years for which there are currently data. This example works for annual data. Minor modifications are needed for the legend if more frequent data are selected above. 

In [ ]:
flowlineId ='06'  # Flowline id to plot
fig, axes = plt.subplots(1, 2, figsize=(14, 7))  # Setup plot
# Loop over glaciers
for glacierId, ax in zip(myFlowlines, axes.flatten()):
    # return interpolated values as vx(time index, distance index)
    vx, vy, vv = myVelSeries.interp(*myFlowlines[glacierId].xykm(), units='km')
    # loop over each profile by time
    for speed, myDate in zip(vv, myVelSeries.time):
        ax.plot(myFlowlines[glacierId].distancekm(), speed, label=myDate.year)
    # pretty up plot
    ax.legend(ncol=2, loc='upper right', fontsize=15)
    ax.set_xlabel('Distance (km)', fontsize=18)
    ax.set_ylabel('Speed (m/yr)', fontsize=18)
    ax.set_title(f'Glacier {glacierId}', fontsize=20)
# For other combinations could have 
for ax in axes.flatten():
    ax.tick_params(axis='both', labelsize=15)
plt.tight_layout()

## Plot Points Through Time

When the map plots were generated above, a set of points 10-k from the start of each flowline was extracted:

In [ ]:
glacierPoints

The time series for each set of points can be plotted as:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 7))
# Loop over glaciers
for glacierId, ax in zip(glacierPoints, axes.flatten()):
    # Loop over flowlines
    for flowlineId in glacierPoints[glacierId]:
        # interpolate to get results vx(time index) for each point
        vx, vy, v = myVelSeries.interp(*glacierPoints[glacierId][flowlineId], units='km')
        ax.plot(myVelSeries.time, v, marker='o',  linestyle='-', color=flowlineColors[flowlineId],label=f'{flowlineId}')
    # pretty up plot
    ax.legend(ncol=3, loc='upper right', title='Flowline ID')
    ax.set_xlabel('year', fontsize=13)
    ax.set_ylabel('Speed (m/yr)', fontsize=13)
    ax.set_title(f'Glacier {glacierId}')
plt.tight_layout()

## Save the Data

While it is convenient to work the date remotely, its nice to be be able to save the data for further processing.

The downloaded subset can be saved in a netcdf and reloaded for to `velSeries` instance for later analysis. 

Note makes sure the data have been subsetted so only the the subset will be saved (~15MB in this example). If not, the entire Greeland data set will be saved (370GB). 

Change `saveData` and `reloadData` below to test this capability.

In [ ]:
saveData = True # Set to True to save data
if saveData:
    myVelSeries.toNetCDF('Glaciers1-2example.nc')

Now open open the file and redo a plot from above with the saved data.

In [ ]:
reloadData = True  # Set to True to reload the saved data
if reloadData:
    fig, axes = plt.subplots(1, 2, figsize=(14, 7))  # Setup plot
    myVelCDF = nisar.nisarVelSeries() # Create Series
    myVelCDF.readSeriesFromNetCDF('Glaciers1-2example.nc')
    #
    for glacierId, ax in zip(myFlowlines, axes.flatten()):
        # return interpolated values as vx(time index, distance index)
        vx, vy, vv = myVelCDF.interp(*myFlowlines[glacierId].xykm(), units='km')
        # loop over each profile by time
        for speed, myDate in zip(vv, myVelSeries.time):
            ax.plot(myFlowlines[glacierId].distancekm(), speed, label=myDate.year)
        # pretty up plot
        ax.legend(ncol=2, loc='upper right', fontsize=15)
        ax.set_xlabel('Distance (km)', fontsize=18)
        ax.set_ylabel('Speed (m/yr)', fontsize=18)
        ax.set_title(f'Glacier {glacierId}', fontsize=20)
# For other combinations could have 
    for ax in axes.flatten():
        ax.tick_params(axis='both', labelsize=15)
    plt.tight_layout()

# Visualizing with widgets

In [ ]:
# Dotson - requires a shapefile to plot that has been formated to lists of lat/lon pairs
# formatting for shapefile: Wcoords2 = [[Wlat2[i],Wlon2[i]] for i in range(Wlat2.shape[0])]

add

center = [-74.23, -112.00]
zoom = 8

mapdt1 = '2022-02-01'

global dc, start, end, file, lon_l, lat_l, lon_r, lat_r, dt

# Pick date
start = widgets.DatePicker(disabled=False)
end = widgets.DatePicker(disabled=False)
# dt = widgets.DatePicker(disabled=False)

# Select from map
draw_control = DrawControl(rectangle={}, polyline={'shapeOptions': {'color': '#fca45d','weight': 1,'opacity': 1.0}}, 
                 polygon={}, circlemarker={})

feature_collection = {
    'type': 'FeatureCollection',
    'features': []}

def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""    
    feature_collection['features'].append(geo_json)

draw_control.on_draw(handle_draw)

m = Map(basemap=basemap_to_tiles(basemaps.NASAGIBS.ModisAquaTrueColorCR, mapdt1),center=center,zoom=zoom)

line1 = Polyline(locations=Wcoords1, weight=1,color='red' , fill=False)
line2 = Polyline(locations=Wcoords2, weight=1,color='red' , fill=False)
m.add_layer(line1)
m.add_layer(line2)

# Provides drawing control - can add something that keeps what is drawn...
m.add_control(draw_control)

# Can upload a shapefile
file = widgets.FileUpload(accept='.shp', multiple=False)

# Specify a bounding box
lon_l = widgets.FloatText(description="lon")
lat_l = widgets.FloatText(description="lat")
lon_r = widgets.FloatText(description="lon")
lat_r = widgets.FloatText(description="lat")

AppLayout(header = VBox([HTML("<h1>Select area (time and space)</h1>"),
                 HBox([Label("Start Date:"), start, 
                       Label("End Date:"), end]),
                 # HBox([Label("Map Date:"), mapdt1])
                        ]),
             center = m,)

We are now going to revisit ATM and ICESat-2 one last time. We have conducted a bias assessment between the two lidars, so now we are going to look at how the land ice heights change over time.

First let's take a look at ATM data from previous years. The CSV file we are going to use is pre-processed L2 data for 2011-2018, much like the data from 2019. These flights are slightly east of the 2019 flight, which was adjusted to better align with ICESat-2.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
# Read in the ATM CSV file
atm_2011_2018 = pd.read_csv('ILATM2_2011_2019_v3.csv')
lltoxy = pyproj.Transformer.from_crs(4326, 3413)

#%matplotlib widget
# Loop through the valid years and plot surface height
years = ['2011', '2013', '2014', '2015', '2016', '2018']
for i,year in enumerate(years):
    lat = atm_2011_2018['Latitude_'+year]
    elev = atm_2011_2018['elev_'+year]
    axes[1].plot(lat, elev, label=year)
#
#
myVelSeries.displayVelForDate('2020-01-01', ax=axes[0], labelFontSize=12, plotFontSize=9, titleFontSize=14, 
                              vmin=1, vmax=3000, units='m', scale='log', midDate=False, colorBarSize='3%', colorBarPosition='bottom')
axes[0].axis('off')
lltoxy = pyproj.Transformer.from_crs(4326, 3413)
for i, year in enumerate(years[3:]):
    lat, lon = atm_2011_2018['Latitude_'+year], atm_2011_2018['Longitude_'+year]
    x, y = lltoxy.transform(lat, lon)
    axes[0].plot(x, y, 'k', linewidth=2)
    v =  myVelSeries.interp(lat, lon, sourceEPSG=4326, returnXR=True).sel(time=datetime(int(year), 6, 1), method='nearest').sel(band='vv').data
    axes[2].plot(lat, v, label=year)
#
axes[1].set_xlabel('latitude')
axes[1].set_ylabel('elevation [m]')
axes[1].legend()
axes[2].set_xlabel('latitude')
axes[2].set_ylabel('Speed [m/yr]')
axes[2].legend()
fig.tight_layout()

It's obvious that there was significant ice loss between 2011 and 2013. The other years appear to be much more variable - let's focus in on the trough at 69.203N...

In [ ]:
# Set the latitude bounds of the surface trough
lat_bounds = [69.1982, 69.2113]

# 2013 has the longest streak of data over this region. We are going to downscale the other years to its length.
lat_2013 = atm_2011_2018['Latitude_2013'][(atm_2011_2018['Latitude_2013']>lat_bounds[0]) & (atm_2011_2018['Latitude_2013']<lat_bounds[1])]

# First, downscale the 2011 data to 2013 resolution
lat = atm_2011_2018['Latitude_2011']
elev = atm_2011_2018['elev_2011'][(lat>lat_bounds[0]) & (lat<lat_bounds[1])].reset_index(drop=True)
        
lat = lat[(lat>lat_bounds[0]) & (lat<lat_bounds[1])].reset_index(drop=True)
spl = splrep(lat[::-1], elev[::-1], s=0)
slp_2011 = splev(lat_2013, spl, der=0)

# Calculate ice loss relative to 2011
delta_h = [0]
std_h = [0]
for i,year in enumerate(years[1:]): # Start loop at 2013 (2012 has no data)
    if year != 2013: # Downscale other years to 2013 resolution
        lat = atm_2011_2018['Latitude_'+year]
        elev = atm_2011_2018['elev_'+year][(lat>lat_bounds[0]) & (lat<lat_bounds[1])].reset_index(drop=True)
        
        # Downscale the data with splines
        lat = lat[(lat>lat_bounds[0]) & (lat<lat_bounds[1])].reset_index(drop=True)
        spl = splrep(lat[::-1], elev[::-1], s=0)
        spl_year = splev(lat_2013, spl, der=0)
        
        # Now calculate the difference relative to 2011
        delta_h.append((spl_year - slp_2011).mean())
        std_h.append((spl_year - slp_2011).std())
    else:
        lat = atm_2011_2018['Latitude_'+year]
        elev = atm_2011_2018['elev_'+year][(lat>lat_bounds[0]) & (lat<lat_bounds[1])].reset_index(drop=True)
        
        # Calculate the difference relative to 2011
        delta_h.append((elev[::-1] - slp_2011).mean())
        std_h.append((spl_year - slp_2011).std())
    
%matplotlib widget
plt.errorbar(years, delta_h, yerr=std_h, marker='.', markersize=12, capsize=4)
plt.xlabel('year')
plt.ylabel('$\Delta$ h [m]')
plt.show()

Ta-da!! Using a few operations, we were able to use ATM data to derive a rough time series of ice sheet elevation change over Jakobshavan.

We're going to try and do the same thing, but for ICESat-2. Because it was launched in late-2018, we are going to try and grab interseasonal measurements from RGT 338 for 2019-2021.

In [ ]:
# Time to go through the icepyx routine again!
import icepyx as ipx

# Specifying the necessary icepyx parameters
short_name = 'ATL06'
lat_bounds = [69.1982, 69.2113]
spatial_extent = [-50, 69.1982, -48.5, 69.2113] # KML polygon centered on Jakobshavan
date_range = ['2019-04-01', '2021-12-30']
rgts = ['338'] # IS-2 RGT of interest

In [ ]:
# Setup the Query object
region = ipx.Query(short_name, spatial_extent, date_range, tracks=rgts)

# Show the available granules
region.avail_granules(ids=True)

In [ ]:
# Set Earthdata credentials
uid = 'zhfair'
email = 'zhfair@umich.edu'
region.earthdata_login(uid, email)

# Order the granules
region.order_granules()

In [ ]:
path = '/home/jovyan/website2022/book/tutorials/DataIntegration/'
region.download_granules(path)

In [ ]:
import h5py

%matplotlib widget

# Iterate through the files to grab elevation, and derive elevation differences relative to April 2019
files = ['processed_' + granule for granule in region.avail_granules(ids=True)[0]]

# Get the initial data from April 2019
with h5py.File(files[0]) as f:
    elev_42019 = f['gt2l/land_ice_segments/h_li'][:]
    lat_42019 = f['gt2l/land_ice_segments/latitude'][:]
    plt.plot(lat_42019, elev_42019, label=files[0][16:24])

delta_h = [0]
std_h = [0]
for file in files[1:]:
    try:
        with h5py.File(file) as f:
            cloud_flag = np.mean(f['gt2l/land_ice_segments/geophysical/cloud_flg_asr'][:])
            # Filter out cloudy scenes
            if cloud_flag < 2:
                lat = f['gt2l/land_ice_segments/latitude'][:]
                elev = f['gt2l/land_ice_segments/h_li'][:]
                date = file[16:24] # Get date of IS-2 overpass
                
                # Find the difference relative to April 2019
                delta_h.append(np.mean(elev - elev_42019))
                std_h.append(np.std(elev - elev_42019))
                
                # Plot the elevation data
                plt.plot(lat, elev, label=date)
    except:
            print('no data')
            
plt.xlabel('latitude')
plt.ylabel('elevation [m]')
plt.legend()
plt.show()

In [ ]:
# Plot the ice sheet change time series
dates = ['20190420','20200117', '20200717', '202110115']
%matplotlib widget
plt.errorbar(dates, delta_h, yerr=std_h, marker='.', markersize=12, capsize=4)
plt.xlabel('date')
plt.ylabel('$\Delta$ h [m]')
plt.show()

There we go! We lost some data due to cloud cover, and the mean change has some spread, but the ICESat-2 data continues to show a downward trend that was suggested by ATM.

## Summary

 🎉 Congratulations! You've completely this tutorial and have seen how we can add  notebook can be formatted, and how to create interactive map visualization with ipyleaflet.
 

```{note}
You may have noticed Jupyter Book adds some extra formatting features that do not necessarily render as you might expect when *executing* a noteook in Jupyter Lab. This "admonition" note is one such example.
```

:::{warning}
Jupyter Book is very particular about [Markdown header ordering](https://jupyterbook.org/structure/sections-headers.html?highlight=headers#how-headers-and-sections-map-onto-to-book-structure) to automatically create table of contents on the website. In this tutorial we are careful to use a single main header (#) and sequential subheaders (#, ##, ###, etc.)
:::

## References

To further explore the topics of this tutorial see the following detailed documentation:

* [Jupyter Book rendering of .ipynb notebooks](https://jupyterbook.org/file-types/notebooks.html)
* [Jupyter Book guide on writing narrative content](https://jupyterbook.org/content/index.html)
* [ipyleaflet documentation](https://ipyleaflet.readthedocs.io)